In [ ]:
'''
The following code computes the transfer map which goes from K2(F) -> K2(E), together with the
projection to the ell-torsion of the Brauer group of E from

    Shmuel Rosset and John Tate. “A reciprocity law for K2-traces”. In: Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.

This code uses some functions from SageMath 9.3.

    Sage Developers. SageMath, the Sage Mathematics Software System (Version 9.3).
    2021. url:https://www.sagemath.org.
'''

def gaussian_elim(M, m):
    '''
    Inputs:
        M - a matrix of the form
                        1 0 0   ...   0 0 0
                        - - -    a    - - -
                        - - -   a^2   - - -
                                 .
                                 .
                                 .
                        - - - a^(d-1) - - -
                        - - -    b    - - -
                        - - -    ab   - - -
                                 .
                                 .
                                 .
                        - - -    b^2  - - -
                                 .
                                 .
                                 .
                        - a^(d-1)b^(m/d-1)-
                        - - -   norm  - - -
        m - the number of rows in the matrix M

    Returns the last row of the matrix M as a linear combination of the first m rows as a vector.
    '''
    I = identity_matrix(M.base_ring(), m+1)
    for j in range(m):
        for i in range(j+1, m+1):
            # Remove all entries of column j past the (j,j) entry.
            I.add_multiple_of_row(i, j, -1*M[i,j])
            M.add_multiple_of_row(i, j, -1*M[i,j])
        for k in range(j+1, m):
            if M[k,j+1]!=0:
                I.swap_rows(j+1, k)
                M.swap_rows(j+1, k)
                I.rescale_row(j+1, 1/M[j+1,j+1])
                M.rescale_row(j+1, 1/M[j+1,j+1])
    I *= -1
    return list(I.submatrix(m, 0, 1, m)[0])


def edit_matrix(mat, editor):
    '''
    Inputs:
        mat - an input matrix
        editor - function to modify each entry in the matrix

    Edits each entry of an input matrix and returns the edited matrix.
    '''
    unwind = list(map(list, list(mat)))
    edited = list(map(lambda vect: list(map(lambda ent: editor(ent), vect)) , unwind))
    return Matrix(edited)


def mult_by_elt(elt, F, Ex, Ex_to_F):
    '''
    Inputs:
        elt - an element in the field F
        F - a field
        Ex - a subfield of F
        Ex_to_F - an embedding from the subfield Ex into F
    
    Returns the matrix associated to multiplication by an element in F, acting on the vector space
    over a subfield Ex.
    '''
    m = F.absolute_degree()/Ex.absolute_degree()
    mat = []
    Fgen = F.0
    Exgen = Ex.0
    ExgeninF = Ex_to_F(Exgen)
    d = Ex.relative_degree()
    mat2 = []

    # Convert to a new basis for F over E as x^i*Fgen^j.
    for i in range(d):
        Exgentothei = ExgeninF^i
        for j in range(m):
            mat2.append((Exgentothei*Fgen^j).list())
    mat2.append(0.list())

    # The matrix mat2 contains as its rows x^i Fgen^j expressed in terms of the old basis.
    # We evaluate the last row of mat2 at an element of F and use gaussian_elim to rewrite the last
    # row of mat2 in terms of a new basis in E.
    for i in range(m):
        z = elt * Fgen^i
        mat2[-1]=z.list()
        Mat2edit = Matrix(mat2).change_ring(Ex)
        matrow = gaussian_elim(Mat2edit, m*d)
        mat.append(matrow)

    # Now that we have elt * Fgen^j using coefficients of a new basis in E, express it in terms
    # of a basis for Ex.
    matwithnewcoeffs = []
    for matrow in mat:
        rownewmat = []
        for j in range(m):
            rownewmat.append(0)
            for i in range(d):
                rownewmat[-1] += ExgeninF^i*Ex_to_F(matrow[i*m+j])
        matwithnewcoeffs.append(rownewmat)
    return matrix(matwithnewcoeffs).transpose()


def mult_by_pol(poly, F, Ex, Ex_to_F, E_pol, E_polT):
    '''
    Inputs:
        poly - a polynomial in x and y over F
        F - a NumberField
        Ex - a subfield of F
        Ex_to_F - an embedding of the subfield Ex into F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_pol_T - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E
    
    Expresses each element in F[x,y] as a sum of monomials in x, y with coefficients in F.
    For each coeff in F, we decompose it as a sum of the basis for F/E. This gives us a
    decomposition of our polynomial in terms of the same basis for F(x,y)/E(x,y).
    
    Returns the matrix associated to multiplication by a polynomial in x and y.
    '''
    M = None
    varname = E_polT.gens()[0]
    for mon in poly.monomials():
        coef = poly.monomial_coefficient(mon)
        Fcoef = F(coef)
        coefmat = mult_by_elt(Fcoef, F, Ex, Ex_to_F)
        
        # Now, the elements of the matrix coefmat lie in E(x). Replace x by T and shift the
        # coefficients to E_polT = (FractionField(E[x,y]))[T]. This is so that we can multiply
        # coefficients by Xs and Ys.
        newcoefmat = edit_matrix(coefmat, lambda ent: E_polT(ent.lift(var=varname)))
        if M == None:
            deg = coefmat.nrows()
            M = matrix(ring=E_polT, nrows=deg, ncols=deg)
        M += E_pol(mon)*newcoefmat
    return M


def Norm_Pol(z, F, Ex, Ex_to_F, E_pol, E_polT):
    '''
    Inputs:
        z - an element in F[x,y]
        F - a Number Field
        Ex - a subfield of F
        Ex_to_F - an embedding of Ex into F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_pol_T - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E

    Returns the determinant of the matrix of multiplication by the element z.
    '''
    M = mult_by_pol(z, F, Ex, Ex_to_F, E_pol, E_polT)
    return M.det()


def Norm(z, F, Ex, Ex_to_F = None, F_pol = None, E_pol=None, E_polT=None):
    '''
    Inputs:
        z - a tuple whose first entry is a list of numerators (each of which is a linear factor, and
            whose second entry is a list of denominators
        F - a field extension of Ex
        Ex - the base field
        Ex_to_F - an embedding of Ex into F
        F_pol - Multivariate Polynomial Ring in x and y over F
        E_pol - Multivariate Polynomial Ring in x and y over E
        E_polT - Univariate Polynomial Ring in T over a Fraction Field of the
                  Multivariate Polynomial Ring in x and y over E

    Returns the field norm of an element over an extension F/Ex.
    '''
    if z in F:
        return F(z).norm(Ex)
    if z in F_pol:
        return Norm_Pol(z, F, Ex, Ex_to_F, E_pol, E_polT)
    znum = z[0]
    zden = z[1]
    nm = 1
    for pol in znum:
        nm *= Norm_Pol(pol, F, Ex, Ex_to_F, E_pol, E_polT)
    for pol in zden:
        nm *= Norm_Pol(pol, F, Ex, Ex_to_F, E_pol, E_polT)^(ell-1)
    return nm


# Remove factors of T from the polynomial, since the reciprocity symbol (f/T)=0.
def star(p, E):
    '''
    Inputs:
        p - a polynomial in T defined over E
        E - a field

    If p(T) = a_n T^n + a_{n-1} T^{n-1} + ... + a_m T^m, returns the polynomial
    p*(T) = (a_m T^m)^{-1} p(T), as defined in
    
    Shmuel Rosset and John Tate. “A reciprocity law for K2-traces”. In: Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.
    '''
    if p in E:
        return 1
    T = p.args()[0]
    pcoeffs = p.coefficients(sparse=False)
    m = 0
    while pcoeffs[m]==0:
        m += 1
    q = p.shift(-m)
    q /= pcoeffs[m]
    return q


def c(p, E):
    '''
    Inputs:
        p - a polynomial in T defined over E
        E - a field

    If p(T) = a_n T^n + a_{n-1} T^{n-1} + ... + a_m T^m, returns c(p) = (-1)^n a_n, as defined in
    
    Shmuel Rosset and John Tate. “A reciprocity law forK2-traces”. In:Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.
    '''
    if p in E:
        return -p
    T = p.args()[0]
    return (-1)^p.degree(T) * p.coefficients()[-1]


def comp_tr(x, y, F, E, F_pol = None, E_pol = None):
    '''
    Inputs:
        x - an element of F
        y - an element of F
        F - an extension of E
        E - a base field
        F_pol - Multivariate Polynomial Ring in x and y over F
        E_pol - Multivariate Polynomial Ring in x and y over E
    
    Computes the transfer map which goes from K2(F) -> K2(E), together with the projection to the ell-torsion
    of the Brauer group of E. Implements the algorithm given in the Proposition of Section 3 in
    
    Shmuel Rosset and John Tate. “A reciprocity law for K2-traces”. In: Comment.Math. Helv.58.1 (1983),
    pp. 38–47.issn: 0010-2571.doi:10.1007/BF02564623.url:https://doi.org/10.1007/BF02564623.
    
    Returns the trace of x and y over the field extension F/E.
    '''
    # If one of x or y is in E, we can simplify the computation.
    if x in E:
        return [(x, Norm(y, F, E))]
    if y in E:
        return [(Norm(x, F, E), y)]

    # Otherwise, we implement the full Rosset-Tate Algorithm.
    # We consider two cases: (1) y is in F(x,y), or (2) y is in F.
    if F_pol == None:
        # Define f and g as polys in E[T] depending on x and y.
        R.<T> = E['T']
        g = x.minpoly(var='T').change_ring(E)
        Ex, Ex_to_F = F.subfield(x, 'Exgen')
        fbar = Norm(y, F, Ex, Ex_to_F)
        f = fbar.lift(var='T')
        E_comb = E
    else:
        E_polF = E_pol.fraction_field()
        F_polF = F_pol.fraction_field()
        E_polT.<T> = E_polF['T']
        F_polT = E_polT.base_extend(FpolF)

        # Get the subfield of F generated by x.
        g = x.minpoly(var='T')
        Ex, Ex_to_F = F.subfield(x, 'Exgen')
        fbar = EpolT(Norm(y, F, Ex, Ex_to_F, F_pol, E_pol, E_polT))
        
        # Define f as fbar reduced to be of minimal degree modulo g.
        _, f = fbar.quo_rem(g)
        E_comb = E_polF
    
    # Recursively compute the polynomials g_i; terminate when g_m is constant.
    glist = [g, f]
    glistar = [star(g, E_comb), star(f, E_comb)]
    while glist[-1] not in E_comb:
        _, newg = glistar[-2].quo_rem(glist[-1])
        glist.append(newg)
        glistar.append(star(newg, E_comb))
    tr = []

    # Recall that tr = -sum(c(star(g_i-1)), c_g) by the Proposition in Section 3 of Rosset-Tate.
    for i in range(1, len(glist)):
        cg_star = c(glistar[i-1], E_comb)
        cg = c(glist[i], E_comb)
        tr.append((cg, cg_star)) # the inverse of the symbol (a, b) is (b, a)
    return tr